<a href="https://colab.research.google.com/github/TBKHori/Music-Recon13/blob/main/Multiple_Inputs_and_Dictionary_Observations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gymnasium
!pip install torch
!pip install stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0


In [8]:
import gymnasium as gym
import torch as th
from torch import nn
from gymnasium import spaces


In [9]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

In [10]:
class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Dict):
        # Call the parent class's constructor to initialize features_dim
        super().__init__(observation_space, features_dim=None)

        # Define extractors for each subspace
        extractors = {
            "image": nn.Sequential(nn.MaxPool2d(4), nn.Flatten()),
            "vector": nn.Linear(observation_space["vector"].shape[0], 16)
        }

        # Compute total concatenated feature size
        total_concat_size = sum(extractor(self._get_dummy_tensor(subspace))
                                for key, extractor in extractors.items())

        self.extractors = nn.ModuleDict(extractors)

        # Update the features dim manually
        self._features_dim = total_concat_size

    def forward(self, observations: th.Tensor) -> th.Tensor:
        encoded_tensor_list = []

        # Extract features for each subspace using their corresponding extractors
        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))

        # Return a (B, self._features_dim) PyTorch tensor, where B is batch dimension.
        return th.cat(encoded_tensor_list, dim=1)

    def _get_dummy_tensor(self, subspace):
        # Helper method to create a dummy tensor for each subspace
        # This is used to compute the size of the extracted features
        dummy_input = th.zeros((1,) + subspace.shape)
        return extractor(dummy_input).flatten().shape[0]